DATA MINING CSC440 - PROJECT
SUBMITTED BY - SAJID HUSSAIN RAFI AHAMED

In [1]:
# import necesarry Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import *
from itertools import *
import time
import math
from random import sample

In [2]:
# Importing the UCI Adult Dataset
columns_names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation",
"relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","salary"]
def data_fetch():
        transaction = list()
        open_text_file = open("adult.data.txt", 'r')
        transaction_counter = 0
        for x in open_text_file:
            transaction_counter = transaction_counter + 1
            tran1 = x.strip().rstrip(',')
            tran1 = tran1.split(', ')
            transaction.append(tran1)
        return transaction, transaction_counter
data,data_count=data_fetch()

In [3]:
#The UCI Dataset contains 32562 data entries spanning over 15 columns
len(data)

32562

In [4]:
# Creation of Dataframe
dataset=pd.DataFrame(data,columns=columns_names,index=None)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
#Dropping rows with Null Values
dataset.dropna(inplace=True)

In [6]:
# Dropping columns with Numerical Values
dataset.drop(["age","fnlwgt","education-num","capital-gain","capital-loss","hours-per-week"],axis=1,inplace=True)

In [7]:
dataset.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,salary
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [8]:
dataset.describe()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,salary
count,32561,32561,32561,32561,32561,32561,32561,32561,32561
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22696,10501,14976,4140,13193,27816,21790,29170,24720


In [9]:
dataset.salary.unique()

array(['<=50K', '>50K'], dtype=object)

In [10]:
dataset.shape
dataset.head()
dataset=dataset.values.tolist()

Apriori Algorithm

In [11]:
class Apriori():

  def __init__(self,df,minimum_support):
    self.data=df # Variable to store dataset
    self.minimum_support=minimum_support # Initialise Minimum support 
    
  def data_fetch(self):  
        transaction = list() 
        dataset = self.data
        transaction_counter = 0
        for x in dataset: # Fetching of transactions row wise from the dataset
            transaction_counter = transaction_counter + 1 #Counter for number of transactions
            tran1 = list(set(x)) # Remove duplicates from transactions
            transaction.append(tran1) 
        return transaction, transaction_counter

  def one_item_counter(self, input_list):
    one_item_set= dict() # Dictionary to store item sets
    for j in input_list:
      for i in j:
        if i in one_item_set:
          one_item_set[i]=one_item_set[i]+1 # Counter to store number of occurences of itemset 
        else:
          one_item_set[i]=1
    return dict(sorted(one_item_set.items(),key = lambda x:x[1],reverse=True))
  
  def generate_l(self,trans_dict,counter): 
      number_of_transaction = counter # Total count of transactions
      trans_to_be_deleted= list()
      for key, value in trans_dict.items(): # Iterate through the item set dictionary
          if float(value/number_of_transaction)<self.minimum_support: # Remove value if less than minimum support
              trans_to_be_deleted.append(key)
          else:
              continue
      for i in trans_to_be_deleted:
          del trans_dict[i]
      return trans_dict

  def return_superset(self,list_trans,k):
      return list(combinations(list_trans,k))  

  def candidate_itemset(self,lk,k):
      candidate_itemset=[]
      supersets = self.return_superset(list(lk.keys()),k) # Generation of candidate keys with K item values 
      for i in supersets:
            candidate_itemset.append(i)
      return candidate_itemset

  def item_counter(self, input_list, candidate_itemset):
        item_set= dict()
        for j in input_list:
            for i in candidate_itemset:
                if set(i).issubset(set(j)):
                    if i in item_set:
                        item_set[i]=item_set[i]+1
                    else:
                        item_set[i]=1
        return dict(sorted(item_set.items(),key = lambda x:x[1],reverse=True))

  def frequent_itemset(self):
        frequent_dict= dict()
        data1,c=self.data_fetch() # Calling Data Fetch function 
        one_item=self.one_item_counter(data1) 
        lk_initial=self.generate_l(one_item,c) # Generate list of items greater than minimum support
        frequent_dict.update(lk_initial) 
        k=2 # Initialise K 
        ck=self.candidate_itemset(lk_initial,k) # Generates Candidate Keys with K=2
        item=self.item_counter(data1,ck)  # Counts the itemsets to generate frequent pattern
        lk=self.generate_l(item,c)
        frequent_dict.update(lk) # Update Dictionary 
        while(len(lk)>0): # Loops through the process for various values of K
          k=k+1
          ck=self.candidate_itemset(lk_initial,k) 
          item=self.item_counter(data1,ck)
          lk=self.generate_l(item,c)
          frequent_dict.update(lk)
        return frequent_dict

In [12]:
dummy_data=data=[['M','O','N','K','E','Y'],['D','O','N','K','E','Y'],
      ['M','A','K','E'],['M','U','C','K','Y'],
      ['C','O','O','K','I','E']]
start_time = time.time()
ap= Apriori(df=dummy_data,minimum_support=0.6)
freq_dict=ap.frequent_itemset()
print(freq_dict)
print("--- %s seconds ---" % (time.time() - start_time))

{'K': 5, 'E': 4, 'O': 3, 'Y': 3, 'M': 3, ('K', 'E'): 4, ('K', 'O'): 3, ('K', 'Y'): 3, ('K', 'M'): 3, ('E', 'O'): 3, ('K', 'E', 'O'): 3}
--- 0.0009942054748535156 seconds ---


In [13]:
start_time = time.time()
ap= Apriori(df=dataset,minimum_support=0.6)
freq_dict=ap.frequent_itemset()
print(freq_dict)
print("--- %s seconds ---" % (time.time() - start_time))

{'United-States': 29170, 'White': 27816, '<=50K': 24720, 'Private': 22696, 'Male': 21790, ('United-States', 'White'): 25621, ('United-States', '<=50K'): 21999, ('White', '<=50K'): 20699, ('United-States', 'Private'): 20135}
--- 0.7739434242248535 seconds ---


Improved Apriori using Partition of transaction

In [14]:
'''Apriori improvement using resampling'''
class Resampled_Apriori():
    
  def __init__(self,df,minimum_support,n):
    self.data=df
    self.minimum_support=minimum_support
    self.count= len(df)
    self.n=n

  def data_fetch(self):
        transaction = list()
        dataset = self.data
        transaction_counter = 0
        for x in dataset:
            transaction_counter = transaction_counter + 1
            tran1 = list(set(x))
            transaction.append(tran1)
        return transaction, transaction_counter
  
  def sampled_data(self,data):
        return (sample(data,self.n))

  def one_item_counter(self, input_list):
    one_item_set= dict()
    for j in input_list:
      for i in list(set(j)):
        if i in one_item_set:
          one_item_set[i]=one_item_set[i]+1
        else:
          one_item_set[i]=1
    return dict(sorted(one_item_set.items(),key = lambda x:x[1],reverse=True))
  
  def generate_l(self,trans_dict,counter):
      number_of_transaction = counter
      trans_to_be_deleted= list()
      for key, value in trans_dict.items():
          if float(value/number_of_transaction)<self.minimum_support:
              trans_to_be_deleted.append(key)
          else:
              continue
      for i in trans_to_be_deleted:
          del trans_dict[i]
      return trans_dict

  def return_superset(self,list_trans,k):
      return list(combinations(list_trans,k))

  def candidate_itemset(self,lk,k):
      candidate_itemset=[]
      supersets = self.return_superset(list(lk.keys()),k)
      for i in supersets:
            candidate_itemset.append(i)
      return candidate_itemset

  def item_counter(self, input_list, candidate_itemset):
        item_set= dict()
        for j in input_list:
            for i in candidate_itemset:
                if set(i).issubset(set(j)):
                    if i in item_set:
                        item_set[i]=item_set[i]+1
                    else:
                        item_set[i]=1
        return dict(sorted(item_set.items(),key = lambda x:x[1],reverse=True))
  
  def frequent_itemset(self):
        frequent_dict= dict()
        data1,c=self.data_fetch()
        data1=self.sampled_data(data1)
        c=len(data1)
        one_item=self.one_item_counter(data1)
        lk_initial=self.generate_l(one_item,c)
        frequent_dict.update(lk_initial)
        k=2
        ck=self.candidate_itemset(lk_initial,k)
        item=self.item_counter(data1,ck)
        lk=self.generate_l(item,c)
        frequent_dict.update(lk)
        while(len(lk)>0):
          k=k+1
          ck=self.candidate_itemset(lk_initial,k)
          item=self.item_counter(data1,ck)
          lk=self.generate_l(item,c)
          frequent_dict.update(lk)
        return frequent_dict

In [15]:
start_time = time.time()
ap2= Resampled_Apriori(df=dummy_data,minimum_support=0.6,n=4)
freq_dict2=ap2.frequent_itemset()
print(freq_dict2)
print("--- %s seconds ---" % (time.time() - start_time))

{'K': 4, 'E': 3, ('K', 'E'): 3}
--- 0.0009968280792236328 seconds ---


In [16]:
start_time = time.time()
ap2= Resampled_Apriori(df=dataset,minimum_support=0.6,n=500)
freq_dict2=ap2.frequent_itemset()
print(freq_dict2)
print("--- %s seconds ---" % (time.time() - start_time))

{'United-States': 446, 'White': 425, '<=50K': 383, 'Private': 358, 'Male': 326, ('United-States', 'White'): 396, ('United-States', '<=50K'): 341, ('White', '<=50K'): 320, ('United-States', 'Private'): 319, ('White', 'Private'): 300}
--- 0.1430070400238037 seconds ---


Fpgrowth Algorithm

In [17]:
class Create_Node():
  def __init__(self,data_point=None,count=1,prev=None, next=None):
    self.data=data_point 
    self.count=count
    self.child={}
    self.prev=prev
    self.next=next
  
  def increase_count(self,n):
    self.count= self.count + n

In [18]:
class Fptree():
  def __init__(self,item_set,value_counts,minimum_support):
    # Initialisation of itemset, Value Counter and minimum support 
    self.item_set=item_set 
    self.value_counts=value_counts
    self.minimum_support=minimum_support

  def item_counter(self):
    item_counter= defaultdict(int) # Creation of Dictionary of integer type
    for index,item in enumerate(self.item_set):
      for x in list(set(item)):
        item_counter[x]+=self.value_counts[index] # counting frequency 
    return item_counter

  def create_header_table(self):
        header_table= self.item_counter()   # Creation of header table
        header_table= self.remove_infrequent_items(header_table)
        return header_table

  def remove_infrequent_items(self,transaction_dict):
      number_of_transaction =len(transaction_dict)
      # Loop through the transactions and delete items below minimum support
      trans_to_be_deleted = [key for key, value in transaction_dict.items() if value<self.minimum_support]
      for i in trans_to_be_deleted:
          del transaction_dict[i]
      return {item:sup for item,sup in sorted(transaction_dict.items(), key=lambda item: item[1], reverse=True)}

  def construct_fptree(self):
     header_table=self.create_header_table()
     if len(header_table)==0: 
       return None, None
     header_table= {i:[header_table[i], None] for i in header_table}
     fp_tree = Create_Node(data_point='Null') # Initialise NULL head node 
     for i, x in enumerate(self.item_set):
       # Update FP Tree for each cleaned and sorted Item set 
        transaction=[]
        for item in x:
          if item in header_table:
            transaction.append(item)
        transaction.sort(key = lambda m: header_table[m][0], reverse=True)
        current_node = fp_tree
        for item in transaction:
          # Update tree with the given item by traversing from root to leaf
            current_node = self.update_fptree(item, current_node, header_table, self.value_counts[i])
     return fp_tree, header_table

  def update_fptree(self, item, node, header_table, child_count):
    if item in node.child:
       # If item already exists, increment the count
        node.child[item].increase_count(child_count)
    else:
      # Create a new branch for the item 
        newnode = Create_Node(data_point=item,count=child_count,prev=node)
        node.child[item] = newnode
        # Link the branch to the header table
        self.update_header_table(item, newnode, header_table)
    return node.child[item]
    
  def update_header_table(self,item, node, header_table):
    if header_table[item][1] is None:
        header_table[item][1] = node
    else:
        left_node = header_table[item][1]
        # Traverse to the last node then link it to the target
        while left_node.next is not None:
            left_node = left_node.next
        left_node.next = node

In [19]:
class Mine_Fptree():
        def __init__(self,header_table,minimum_support,prefix, frequent_item_sets ):
          self.header_table=header_table
          self.minimum_support=minimum_support
          self.prefix=prefix
          self.frequent_item_sets= frequent_item_sets

        def mine_tree(self,header_table,prefix,frequent_item_sets):
            #Create a list with Sorted items with frequency 
            sorted_List = [k for k, v in sorted(header_table.items(), key=lambda item: item[1][0])]
            for item in sorted_List:
              # Keep adding suffix patterns with frequent patterns generated from conditional FP-tree
                new_freq_itemset = prefix.copy()
                new_freq_itemset.add(item)
                frequent_item_sets.append(new_freq_itemset)
                 # Find all prefix path, construct conditional pattern base
                conditional_pattern, frequencies = self.find_all_prefixpaths(item,header_table)
                new_Fptree=Fptree(conditional_pattern, frequencies, self.minimum_support)
                # Construct conditonal FP Tree with conditional pattern base
                conditional_fp_tree, new_fptable = new_Fptree.construct_fptree()
                if new_fptable != None:
                  #Keep mining recursively on the tree
                    self.mine_tree(new_fptable, new_freq_itemset, frequent_item_sets)
                    
        def find_all_prefixpaths(self,item, header_table):
            current_node = header_table[item][1] # Initial node in the linked list
            conditional_pattern = []
            frequencies = []
            while current_node != None:
                prefixpath = []
                # Traverse from leaf upto the root
                self.ascend_fptree(current_node, prefixpath)
                if len(prefixpath) > 1:
                    conditional_pattern.append(prefixpath[1:]) # to store prefix path 
                    frequencies.append(current_node.count) # to store count of prefix path
                current_node = current_node.next #point to next node
            return conditional_pattern, frequencies
                    
        def ascend_fptree(self,node, prefixpath):
            if node.prev != None:
                prefixpath.append(node.data)
                self.ascend_fptree(node.prev, prefixpath)

In [20]:
minimum_support = 0.6*len(dummy_data)

In [21]:
start_time = time.time()
FP= Fptree(dummy_data,[1]*len(dummy_data),minimum_support)
fptree, header_table = FP.construct_fptree()
freq_items_in_transaction = []
Mining= Mine_Fptree(header_table, minimum_support, set(), freq_items_in_transaction )
Mining.mine_tree(header_table, set(),freq_items_in_transaction)
print(freq_items_in_transaction)
print("--- %s seconds ---" % (time.time() - start_time))

[{'O'}, {'O', 'K'}, {'E', 'O', 'K'}, {'E', 'O'}, {'Y'}, {'K', 'Y'}, {'M'}, {'M', 'K'}, {'E'}, {'E', 'K'}, {'K'}]
--- 0.0009963512420654297 seconds ---


In [22]:
minimum_support = 0.6*len(dataset)

In [23]:
start_time = time.time()
FP= Fptree(dataset,[1]*len(dataset),minimum_support)
fptree, header_table = FP.construct_fptree()
freq_items_in_transaction = []
Mining= Mine_Fptree(header_table, minimum_support, set(), freq_items_in_transaction )
Mining.mine_tree(header_table, set(),freq_items_in_transaction)
print(freq_items_in_transaction)
print("--- %s seconds ---" % (time.time() - start_time))

[{'Male'}, {'Private'}, {'Private', 'United-States'}, {'<=50K'}, {'<=50K', 'White'}, {'<=50K', 'United-States'}, {'White'}, {'United-States', 'White'}, {'United-States'}]
--- 0.24140548706054688 seconds ---


In [24]:
time_taken=[]
min_sup= np.linspace(0.2, 1.0, num=5)
for i in min_sup:
    start_time = time.time()
    ap= Apriori(df=dataset,minimum_support=i)
    freq_dict=ap.frequent_itemset()
    time_taken.append((time.time() - start_time))
    

In [25]:
time_taken2=[]

start_time = time.time()
FP= Fptree(dataset,[1]*len(dataset),0.2*len(dataset))
fptree, header_table = FP.construct_fptree()
freq_items_in_transaction = []
Mining= Mine_Fptree(header_table, minimum_support, set(), freq_items_in_transaction )
Mining.mine_tree(header_table, set(),freq_items_in_transaction)
print(freq_items_in_transaction)
time_taken2.append((time.time() - start_time))

[{'Some-college'}, {'>50K'}, {'Not-in-family'}, {'HS-grad'}, {'Never-married'}, {'Female'}, {'Husband'}, {'Married-civ-spouse'}, {'Male'}, {'Private'}, {'Private', 'United-States'}, {'<=50K'}, {'<=50K', 'White'}, {'<=50K', 'United-States'}, {'White'}, {'United-States', 'White'}, {'United-States'}]


In [26]:
start_time = time.time()
FP= Fptree(dataset,[1]*len(dataset),0.4*len(dataset))
fptree, header_table = FP.construct_fptree()
freq_items_in_transaction = []
Mining= Mine_Fptree(header_table, minimum_support, set(), freq_items_in_transaction )
Mining.mine_tree(header_table, set(),freq_items_in_transaction)
print(freq_items_in_transaction)
time_taken2.append((time.time() - start_time))

[{'Husband'}, {'Married-civ-spouse'}, {'Male'}, {'Private'}, {'Private', 'United-States'}, {'<=50K'}, {'<=50K', 'White'}, {'<=50K', 'United-States'}, {'White'}, {'United-States', 'White'}, {'United-States'}]


In [27]:
start_time = time.time()
FP= Fptree(dataset,[1]*len(dataset),0.6*len(dataset))
fptree, header_table = FP.construct_fptree()
freq_items_in_transaction = []
Mining= Mine_Fptree(header_table, minimum_support, set(), freq_items_in_transaction )
Mining.mine_tree(header_table, set(),freq_items_in_transaction)
print(freq_items_in_transaction)
time_taken2.append((time.time() - start_time))

[{'Male'}, {'Private'}, {'Private', 'United-States'}, {'<=50K'}, {'<=50K', 'White'}, {'<=50K', 'United-States'}, {'White'}, {'United-States', 'White'}, {'United-States'}]


In [28]:
start_time = time.time()
FP= Fptree(dataset,[1]*len(dataset),0.8*len(dataset))
fptree, header_table = FP.construct_fptree()
freq_items_in_transaction = []
Mining= Mine_Fptree(header_table, minimum_support, set(), freq_items_in_transaction )
Mining.mine_tree(header_table, set(),freq_items_in_transaction)
print(freq_items_in_transaction)
time_taken2.append((time.time() - start_time))

[{'White'}, {'United-States', 'White'}, {'United-States'}]


In [29]:
time_taken

[150.9249587059021,
 3.0856943130493164,
 0.707111120223999,
 0.17855381965637207,
 0.15056347846984863]

In [30]:
time_taken2

[0.5565123558044434,
 0.3131685256958008,
 0.2991957664489746,
 0.24335074424743652]

In [31]:
min_sup

array([0.2, 0.4, 0.6, 0.8, 1. ])